In [6]:
from IPython.display import Image, FileLink
import pandas as pd
import pandas.rpy.common as com

/opt/anaconda/lib/python2.7/site-packages/pandas/rpy/__init__.py:11: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  FutureWarning)


In [7]:
%load_ext rpy2.ipython

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [8]:
%%R
library(data.table)
library(dplyr)
library(magrittr)
library(tidyr)

In [9]:
%%R
FDR = 0.10
df.l2fc = read.csv("../data/l2fc.csv", stringsAsFactors = FALSE) %>%
    mutate(Day = ifelse(Day == 1, 12, ifelse(Day == 2, 82, 0))) %>%
    filter(padj <= FDR, l2FC >= 1, Amdmt == 2) %>% 
    group_by(OTU) %>%
    summarize(`Day of Response` = paste(Day, collapse = ", "))

In [10]:
%%R
df.l2fc %>% head

Source: local data frame [6 x 2]

       OTU Day of Response
     (chr)           (chr)
1   OTU.10          12, 82
2  OTU.100              12
3 OTU.1008              82
4 OTU.1013              82
5 OTU.1016              82
6 OTU.1022          12, 82


In [11]:
%%R
df.blast = tbl_df(read.table("../../data/tmp/blast_out_otusn_LTP115.m6",
                  sep = "\t"))
names(df.blast) = c("OTU", "acc", "pid", "alnlen", "mismatches", "gaps", 
                    "qstart", "qend", "sstart", "send", "evalue", "bit")

df.blast

Source: local data frame [3,418,929 x 12]

      OTU      acc   pid alnlen mismatches  gaps qstart  qend sstart  send
   (fctr)   (fctr) (dbl)  (int)      (int) (int)  (int) (int)  (int) (int)
1   OTU.1 AB258386 96.25    373         14     0      1   373    459   831
2   OTU.1 AB021492 95.73    375         12     4      1   373    471   843
3   OTU.1 JN591314 95.73    375         12     4      1   373    471   843
4   OTU.1 AY769083 95.71    373         16     0      1   373    459   831
5   OTU.1 AB264131 95.71    373         16     0      1   373    439   811
6   OTU.1 EF530202 95.73    375         12     4      1   373    445   817
7   OTU.1 AB245347 95.71    373         16     0      1   373    455   827
8   OTU.1 AB277583 95.45    374         16     1      1   373    440   813
9   OTU.1 EU707560 95.17    373         18     0      1   373    428   800
10  OTU.1 EU682685 95.17    373         18     0      1   373    470   842
..    ...      ...   ...    ...        ...   ...    ...  

In [12]:
%%R
df.tax = tbl_df(read.csv("/var/seq_data/silva/silva_blastdb/full_names.csv"))
df.tax

Source: local data frame [1,426,450 x 2]

        acc                           full_name
     (fctr)                              (fctr)
1  AX003092          Enterococcus casseliflavus
2  AX044029              Neisseria meningitidis
3  EU271959            Myzocytiopsis intermedia
4  AX039535            Dehalococcoides mccartyi
5  EU273602                   Acorus americanus
6  AX175616                Marinomonas communis
7  AX044033              Neisseria meningitidis
8  AB000389         Pseudoalteromonas elyakovii
9  EU271960          Myzocytiopsis sp. venatrix
10 AB001439 Pseudomonas syringae pv. actinidiae
..      ...                                 ...


In [13]:
%%R
df.blast = left_join(df.blast, df.tax)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Joining by: "acc"

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [14]:
%%R
df.blast = left_join(df.blast, df.l2fc)

/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Joining by: "OTU"

  res = super(Function, self).__call__(*new_args, **new_kwargs)


In [15]:
%%R
df.blast

Source: local data frame [3,559,283 x 14]

     OTU      acc   pid alnlen mismatches  gaps qstart  qend sstart  send
   (chr)    (chr) (dbl)  (int)      (int) (int)  (int) (int)  (int) (int)
1  OTU.1 AB258386 96.25    373         14     0      1   373    459   831
2  OTU.1 AB021492 95.73    375         12     4      1   373    471   843
3  OTU.1 JN591314 95.73    375         12     4      1   373    471   843
4  OTU.1 AY769083 95.71    373         16     0      1   373    459   831
5  OTU.1 AB264131 95.71    373         16     0      1   373    439   811
6  OTU.1 EF530202 95.73    375         12     4      1   373    445   817
7  OTU.1 AB245347 95.71    373         16     0      1   373    455   827
8  OTU.1 AB277583 95.45    374         16     1      1   373    440   813
9  OTU.1 EU707560 95.17    373         18     0      1   373    428   800
10 OTU.1 EU682685 95.17    373         18     0      1   373    470   842
..   ...      ...   ...    ...        ...   ...    ...   ...    ...  

In [16]:
%%R
df.top.hits = df.blast %>%
    group_by(OTU) %>%
    mutate(bit.rank = rank(desc(bit), ties.method = "min")) %>%
    filter(bit.rank == 1) %>%
    group_by()

In [17]:
%%R
df.top.hits %>% head

Source: local data frame [6 x 15]

    OTU      acc    pid alnlen mismatches  gaps qstart  qend sstart  send
  (chr)    (chr)  (dbl)  (int)      (int) (int)  (int) (int)  (int) (int)
1 OTU.1 AB258386  96.25    373         14     0      1   373    459   831
2 OTU.2 AY624135 100.00    372          0     0      1   372    473   844
3 OTU.2 AF208513 100.00    372          0     0      1   372    467   838
4 OTU.2 GU433448 100.00    372          0     0      1   372    423   794
5 OTU.2 AB300992 100.00    372          0     0      1   372    452   823
6 OTU.2 HQ231463 100.00    372          0     0      1   372    418   789
Variables not shown: evalue (dbl), bit (dbl), full_name (fctr), Day of Response
  (chr), bit.rank (int)


In [18]:
%%R
FDR = 0.10

df.table = df.top.hits %>%
    filter(!is.na(`Day of Response`)) %>%
    group_by(OTU) %>%
    summarize(hits = paste(full_name, collapse = "|"), 
              pid = first(pid), 
              `Day of Response` = first(`Day of Response`))

In [19]:
df_table = com.load_data("df.table")

In [20]:
df_table.head()

,OTU,hits,pid,Day of Response
1,OTU.10,Terrabacter tumescens|Terrabacter sp. ON10|Ter...,100.00,"12, 82"
2,OTU.100,Novosphingobium resinovorum|Novosphingobium pe...,99.46,12
3,OTU.1008,Phaselicystis flava,92.25,82
4,OTU.1013,Ilumatobacter fluminis,94.15,82
5,OTU.1016,Sphingomonas changbaiensis,96.25,82


In [21]:
df_table["OTU_num"] = [int(i.split(".")[1]) for i in df_table["OTU"]]

In [22]:
df_table.sort(columns=["OTU_num"],inplace=True)

In [23]:
def list_genera(l):
    if len(l.split("|")) > 10:
        names = l.split("|")
        names_abbr = []
        seen = []
        for name in names:
            genus, species = name.split(" ", 1)
            if genus in seen:
                continue
            else:
                seen.append(genus)
                names_abbr.append("\mbox{" + "\\textit{" + genus + " spp." + "}" + "}")
        return ", ".join(names_abbr) 
    elif len(l.split("|")) > 1:
        return  ", ".join(["\mbox{" + "\\textit{" + i + "}" + "}" for i in set(l.split("|"))])
    else:
        return "\mbox{" + "\\textit{" + l + "}" + "}"
    
def get_latex(row):
    latex = ""
    OTU = row["OTU"]
    pid = row["pid"]
    Day = row["Day of Response"]
    
    if pid >= 90:
        genera = list_genera(row["hits"])
    else:
        genera = "{No hits of at least 90\% identity}"

    #tax = " ".join(["\mbox{" + "\\textit{" + i + "}" + "}" for i in row[["Phylum","Class","Order"]]]).replace("_","-")
    
    #l2fc = pd.Series(row["log2FoldChange"]).round(2).values[0]
    #acc = df.acc.iloc[0]
    return OTU + " & " + str(Day) + " & " + genera + " & " + str(pid) + r" \\ " + r"\midrule" + "\n"

In [42]:
s = df_table.apply(get_latex, axis=1)

table_framework = r"""
\documentclass[10pt]{article}
\usepackage{multirow, array, booktabs, longtable, threeparttablex}
\usepackage{array}
\newcolumntype{P}[1]{>{\raggedright\arraybackslash}p{#1}}

\usepackage{geometry} 
\geometry{tmargin=1in, bmargin=1in, lmargin=0.5in, rmargin=0.5in} 

\begin{document}

\setcounter{table}{1}

\thispagestyle{empty}

\begin{ThreePartTable}
\begin{TableNotes}
\item[a] $log_{2}$ fold change greater than 1, adjusted P-value less than 0.10. 
\end{TableNotes}

\begin{longtable}{llP{7cm}l}

\caption{OTUs enriched in stover amended soil relative to control BLAST against Living Tree Project} \\
\toprule 
    \textbf{OTU ID} & 
    \textbf{Day of Response} \tnote{a} & 
    \textbf{Top BLAST hits} & 
    \textbf{BLAST \%%ID} \\
\midrule
\endfirsthead

\multicolumn{3}{c}
{{\tablename\ \thetable{} -- continued from previous page}} \\
\midrule
    \textbf{OTU ID} & 
    \textbf{Day of Response} \tnote{b} & 
    \textbf{Top BLAST hits} & 
    \textbf{BLAST \%%ID} \\
\midrule
\endhead
    %s
\bottomrule
\insertTableNotes
\end{longtable}

\end{ThreePartTable}
 
\end{document}"""%"".join(s.values)

with open("../../data/tmp/table_2.tex", "w") as out:
    out.write(table_framework)

In [43]:
!head ../../data/tmp/table_2.tex


\documentclass[10pt]{article}
\usepackage{multirow, array, booktabs, longtable, threeparttablex}
\usepackage{array}
\newcolumntype{P}[1]{>{\raggedright\arraybackslash}p{#1}}

\usepackage{geometry} 
\geometry{tmargin=1in, bmargin=1in, lmargin=0.5in, rmargin=0.5in} 

\begin{document}


In [44]:
s.shape

(671,)

In [45]:
!latex ../../data/tmp/table_2.tex >/dev/null
!dvipdf table_2.dvi LTP_blast_2.pdf

In [46]:
FileLink("LTP_blast_2.pdf")

/home/thea/Field/PyOM/manuscript_figures/LTP_blast_2.pdf